In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os

In [2]:
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [3]:
class_name_to_id_mapping = {"no_mask": 0,
                           "yes_mask": 1}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_text(info_dict, annotation_path, save_file_name):
    print_buffer = [annotation_path.replace("Annotations", "Images") + info_dict["filename"]]
    
    # For each bounding box
    for b in info_dict["bboxes"]:      
        xmin = b["xmin"]
        ymin = b["ymin"]
        xmax = b["xmax"]
        ymax = b["ymax"]
        if b["class"] == "no_mask":
            class_id = 0
        else:
            class_id = 1
        
        #Write the bbox details to the file 
        print_buffer.append(" {},{},{},{},{}".format(xmin, ymin, xmax, ymax, class_id))
    
    # Save the annotation to disk
    with open(save_file_name, "a") as f:
        for lines in print_buffer:
            f.write(lines)
        f.write("\n")

In [4]:
from tqdm import tqdm
annotation_path = ["./train/Annotations/", "./eval/Annotations/", "./test/Annotations/"]

for path in annotation_path:
    annotations = [os.path.join(path, x) for x in os.listdir(path) if x[-3:] == "xml"]
    
    save_file_name = path.split("/")
    save_file_name = f"{save_file_name[1]}_text.txt"
    with open(save_file_name, "w") as f:
        pass

    for ann in tqdm(annotations):
        info_dict = extract_info_from_xml(ann)
        convert_to_text(info_dict, path, save_file_name)
        

100%|██████████| 29/29 [00:00<00:00, 2216.78it/s]
